## **Syntactic sugar in nn.Module** 

Example of **syntactic sugar** in Python using **special (dunder) methods**.

```python
model = MyModel()
output = model(input_tensor)
```

Even though it looks like `model` is a function being called, you're actually leveraging syntactic sugar via Python’s `__call__` method.


When you call an instance like a function:

```python
output = model(input_tensor)
```

Python automatically calls:

```python
output = model.__call__(input_tensor)
```

In `torch.nn.Module`, the base class defines `__call__` like this:

```python
def __call__(self, *args, **kwargs):
    return self.forward(*args, **kwargs)
```

So the `forward()` method gets called when you use function-call syntax on the object.

---

**Example:**

```python
import torch
import torch.nn as nn

class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(10, 1)

    def forward(self, x):
        return self.linear(x)

model = MyModel()

# This actually calls model.__call__(x), which calls model.forward(x)
x = torch.randn(5, 10)
output = model(x)
```
---



## **`nn.Parameter`**
In PyTorch, `nn.Parameter` is used to **mark a tensor as a learnable parameter** within a `nn.Module`. This tells PyTorch that the tensor should be:

1. **Registered as a parameter** (visible in `model.parameters()`).
2. **Updated by optimizers** during training (when `.backward()` is called).

---

###  When to Use `nn.Parameter`

Use `nn.Parameter` when:

* You want a custom tensor (not from a built-in layer like `nn.Linear`) to be **learned** during training.
* You define a module and want PyTorch to **automatically track** its parameters.

---

###  When Not to Use It

* For constant values or buffers (use `self.register_buffer` instead).
* When the tensor should **not** be updated by the optimizer.

---

### Basic Example

In [1]:
import torch
import torch.nn as nn

class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        # A learnable scalar parameter initialized to 1.0
        self.alpha = nn.Parameter(torch.tensor(1.0))

    def forward(self, x):
        return self.alpha * x

model = MyModel()
print(list(model.parameters()))  # Shows alpha as a learnable parameter


[Parameter containing:
tensor(1., requires_grad=True)]


In [4]:
class CustomLinear(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(out_features, in_features))
        self.bias = nn.Parameter(torch.randn(out_features))

    def forward(self, x):
        return x @ self.weight.T + self.bias
        
x = torch.randn(10, 3)
model = CustomLinear(3, 2)

optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

for _ in range(5):
    out = model(x)
    loss = out.sum()
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

print(model.weight)  # It's being updated!


Parameter containing:
tensor([[ 0.9887, -0.2748, -0.8510],
        [-0.0530,  1.7576,  0.9672]], requires_grad=True)
